In [ ]:
import pandas as pd
from tqdm import tqdm

df1 = pd.read_csv('./Data/1_MatDX/MatDX_nomad_EF_Binary.csv')
df2 = pd.read_csv('./Data/1_MatDX/MatDX_nomad_EF_Ternary.csv')

# Concatenate the DataFrames
df = pd.concat([df1, df2], ignore_index=True)

In [14]:
dict = {}

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
    for element in eval(row['formation_energy'])['reference'].keys():
        entry_id = eval(row['formation_energy'])['reference'][element].split('/')[-1]
        dict[element] = entry_id

Processing Rows: 100%|██████████| 20000/20000 [00:01<00:00, 13472.53it/s]


In [15]:
print(dict)

{'Na': 'fZBjCAvL-C0KpykVeDkglk4DiMJE', 'Ir': 'Jxd3Gq7w7vgx3vAf4QaOcIUrNIvf', 'As': 'kWw6gjhjpszGSs-lJVXHTwQkY4RP', 'Pt': 'hfTXu5BPKf83TlYNvYL_xjc7wecf', 'Ni': '2z6qKgNMlPjxK4R1fKqhhgll26lp', 'Ta': 'Jno4RsivxTTSMHe4wn8p4qW8Ef6G', 'Mo': 'TwiQxc0usHCKW3MtmHjbnh8mT5GC', 'Sm': '_QRJJknIDoDicf-YE5iLlQPtC5ne', 'P': 'VliAGY_Z2yfFnSNW4d80E-Y7rLrP', 'Ge': '2eVSxv5n6du3VVsS6uZHLFviKxDn', 'Mn': '8IffBWYWeF2VvGN-TtippJCbAlST', 'Sb': 'e0MYdRBQr9wEXHzKsLol9DZ8gMvs', 'Br': '7DmDJRePJNTiaaJiAv-YuiT0_OyO', 'Ba': 'Lj0gGgNhl0ZMZK5-5xnXiYpf0ut5', 'Ca': 'c08VF8Y4GAg0AFj1CQdui2kVY_IF', 'Pd': 'B_YsxQ9yjPSLQPBbgi6ptP2Q1A64', 'Sr': '9CyQcCUbCgpVbqUi9LKvMkTBuex3', 'Fe': 'uv45-FAwcp05wlvr2QydJ3AkRZeu', 'Pb': 'aYJtYKNr85nM-0acWcW3IEbYkJ65', 'In': 'jidMzp8f26ns8Jif2fj8Gy8eG6id', 'Sn': 'OAw0poj5HN3ZrB-djDMlDbA7-aNL', 'Nb': '861X4hlRjiSg-bvBzuNmZtnnHOYr', 'Bi': 'j4YzvDGcNGpWqp9v6XPFHS3dF2zw', 'Ga': 'vRXRCHTc1BwC0KVL2zTyDtxvDQO7', 'Cu': 'uVTSZjDVQgSURDhx2j8aTT9Zvd6S', 'W': 'dcAjiz1dsca03eeBAM-fTgQjR8M_', 'Rh': '7pAP54

In [16]:
import requests
import numpy as np
from ase import Atoms
# from mace.calculators import mace_mp
# calc = mace_mp(model="medium", dispersion=False, default_dtype='float64', device='cpu')
from sevenn.sevennet_calculator import SevenNetCalculator
sevennet_0_cal = SevenNetCalculator("7net-0", device='cpu', default_dtype='float64')  # 7net-0, SevenNet-0, 7net-0_22May2024, 7net-0_11July2024 ...

In [17]:
E_ref = {}

for key in dict.keys():
    print(key)
    entry_id = dict[key]
    url = f"https://nomad-lab.eu/prod/rae/api/v1/entries/{entry_id}/archive"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        
        # Navigate to the relevant structure data
        try:
            idealized_structure = data['data']['archive']['section_metadata']['encyclopedia']['material']['idealized_structure']
            
            # Extract necessary fields
            atom_labels = idealized_structure['atom_labels']
            atom_positions = idealized_structure['atom_positions']
            lattice_vectors = idealized_structure['lattice_vectors']

            # Convert to numpy arrays if not already in that format
            positions = np.array(atom_positions)
            cell = np.array(lattice_vectors) * 1e10

            # Create the ASE Atoms object
            refs = Atoms(symbols=atom_labels, scaled_positions=positions, cell=cell, pbc=True)
            # refs.calc = calc
            
            # E_mace = refs.get_potential_energy()

            refs.calc = sevennet_0_cal

            E_7net = refs.get_potential_energy()

            E_ref[key] = [len(refs), E_7net]
        except KeyError as e:
            print(f"KeyError: Missing expected key in the data structure - {e}")

    else:
        print(f"Error: {response.status_code}")


Na
Ir
As
Pt
Ni
Ta
Mo
Sm
P
Ge
Mn
Sb
Br
Ba
Ca
Pd
Sr
Fe
Pb
In
Sn
Nb
Bi
Ga
Cu
W
Rh
V
Ac
I
Eu
U
Cr
Ce
Rb
K
Ag
Al
Yb
Cs
Au
Pa
Th
Np


In [18]:
E_ref

{'Na': [2, -2.606849193572998],
 'Ir': [4, -35.44403076171875],
 'As': [6, -27.838623046875],
 'Pt': [4, -24.23782730102539],
 'Ni': [4, -23.073667526245117],
 'Ta': [2, -23.67586326599121],
 'Mo': [2, -21.368026733398438],
 'Sm': [3, -12.97471809387207],
 'P': [8, -42.978050231933594],
 'Ge': [8, -36.62063980102539],
 'Mn': [58, -528.6471557617188],
 'Sb': [6, -24.639816284179688],
 'Br': [8, -12.40904426574707],
 'Ba': [2, -3.647536277770996],
 'Ca': [4, -7.968954086303711],
 'Pd': [4, -20.746471405029297],
 'Sr': [4, -6.70284366607666],
 'Fe': [2, -16.729881286621094],
 'Pb': [4, -14.768635749816895],
 'In': [2, -5.430088043212891],
 'Sn': [2, -7.895471572875977],
 'Nb': [2, -20.185810089111328],
 'Bi': [4, -15.2405366897583],
 'Ga': [8, -24.179044723510742],
 'Cu': [4, -16.384796142578125],
 'W': [2, -25.5202693939209],
 'Rh': [4, -29.364112854003906],
 'V': [2, -18.162288665771484],
 'Ac': [4, -16.342832565307617],
 'I': [8, -11.695618629455566],
 'Eu': [2, -20.464466094970703],
 

In [19]:
import json

# Specify the file name
file_name = "reference_7net.json"

# Save the dictionary as a JSON file
with open(file_name, 'w') as json_file:
    json.dump(E_ref, json_file, indent=4)  # indent=4 makes the JSON pretty-printed

print(f"Dictionary saved as {file_name}")

Dictionary saved as reference_7net.json
